In [19]:
import math
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import os

import matplotlib.pyplot as plt


def expectation_maximization(dataset, k, eps, max_iteration=50):
    """
    Expectation Maximization algorithm implementation from chapter 13
    :param dataset:
    :param k:
    :param eps:
    :return:
    """
    d = dataset.shape[1]
    t = 0
    sigmai = np.identity(d)
    ck, mus, sigmas = init_para(k, d, sigmai, dataset)

    wij = np.array(np.zeros((k, dataset.shape[0])))

    while True:
        t += 1
        wij, ck_new, mus_new, sigmas_new = perform_iteration(dataset, ck, mus, sigmas)
        print("mus:", mus)
        if t > max_iteration or (t > 1 and get_diff_mus(mus_new, mus, k, d) <= eps):
            break
        ck = ck_new
        mus = mus_new
        sigmas = sigmas_new
    print("Final Mus:", mus)
    print("Final Covariance:", sigmas)
    print("No of. Iterations:", t)
    df = pd.DataFrame(wij)
    df["cluster"] = df.apply(lambda row: find_max_prob(row), axis=1)
    print(df)
    print(df.groupby(by="cluster").count())

    ax = sns.distplot(df.cluster, color = 'red')



def find_max_prob(row):
    max = row.max()
    for i in range(len(row)):
        if max == row[i]:
            return i + 1


def get_diff_mus(mus_new, mus, k, d):
    sum = 0
    for i in range(k):
        diffij = mus_new[i] - mus[i]
        sum += diffij.sum() ** 2
    return sum


def init_para(k, d, sigmai, dataset):
    ck = np.array(np.ones((k, 1))) * (1 / float(k))
    mus = initMus(d, k, dataset)
    sigmas = []
    for i in range(k):
        sigmas.append(sigmai)
    return ck, mus, sigmas


def calculate_cluster_probability(X, mu, variance):
    dim = np.shape(X)[0]
    # print dim
    pi = math.pi
    exp = np.exp

    X = np.mat(X)
    mu = np.mat(mu)
    tmp = X - mu

    variance = np.mat(variance)
    # print variance
    deter = np.linalg.det(variance)
    # deter = 1
    tmp0 = np.linalg.pinv(variance)
    tmp1 = tmp * tmp0 * tmp.transpose()
    # print tmp0,tmp1,deter,dim
    prob = 1 / (((2 * pi) ** (dim / 2)) * (deter ** 0.5)) * exp(-0.5 * tmp1)
    # print prob
    return prob


def perform_iteration(X, pi, mu, sigma):
    num, dim = np.shape(X)
    c, notused = np.shape(pi)
    k = len(mu)
    Nk = np.array(np.zeros((k, 1)))
    wij = np.array(np.zeros((num, k)))
    sigma_new = []
    mu_new = []
    pi_new = np.array(np.zeros((c, 1)))
    for i in range(num):
        total = 0
        for j in range(k):
            wij[i, j] = pi[j] * calculate_cluster_probability(X[i], mu[j], sigma[j])
            total += wij[i, j]
        for j in range(k):
            wij[i, j] = wij[i, j] / total
    # print 'wij',wij,'\n'

    for j in range(k):
        tmp_mu = np.array(np.zeros((1, dim)))
        #
        for i in range(num):
            Nk[j, 0] += wij[i, j]
            tmp_mu += wij[i, j] * X[i]
        mu_new.append(tmp_mu / Nk[j, 0])
    # print 'mu_new',mu_new,'\n'

    for j in range(k):
        tmp_var = np.array(np.zeros((dim, dim)))
        for i in range(num):
            tmp_var += wij[i, j] * (
                    (X[i] - mu_new[j]).transpose() * (X[i] - mu_new[j]))
        sigma_new.append(tmp_var / Nk[j, 0])
    # print 'sigma_new',sigma_new,'\n'

    for j in range(k):
        pi_new[j] = Nk[j] / num
    # print 'pi_new',pi_new,'\n'

    return wij, pi_new, mu_new, sigma_new


def initMus(d, k, dataset):
    """
    Initialize mus(centroids for clusters) for k clusters with d dimension for dataset
    :param d:
    :param k:
    :param dataset:
    :return: mus as dataframe
    """
    data = pd.DataFrame(dataset)
    mins = data.min()
    maxs = data.max()
    mus = []
    for i in range(d):
        mus.append([xs for xs in np.random.uniform(mins[i], maxs[i], k)])
    mu = pd.DataFrame(mus)
    mu = mu.transpose()
    return np.array(mu)


def main():
    dataset = pd.read_csv("iris.csv", header=None, names=["x1", "x2", "x3", "x4", "x5"])
    dataset.drop(labels=["x5"], inplace=True, axis=1)
    expectation_maximization(np.array(dataset), 3, 0.001)

    # dataset = np.array([[1.0], [1.3], [2.2], [2.6], [2.8], [5.0], [7.3], [7.4], [7.5], [7.7], [7.9]])
    # expectation_maximization(dataset, 2, 0.0001)

main()


mus: [[5.10737602 3.51187247 5.82051001 0.49197172]
 [6.92564625 2.11283532 6.75341085 2.285156  ]
 [5.64072562 2.22947292 6.48628155 0.93307126]]
mus: [array([[5.40327468, 3.14752838, 2.75493741, 0.77284046]]), array([[6.77054435, 3.01730366, 5.56740004, 1.98211516]]), array([[6.13358961, 2.82930634, 4.73188152, 1.59517875]])]
mus: [array([[5.31128062, 3.17281881, 2.50377061, 0.6638433 ]]), array([[6.74197873, 3.02806929, 5.56080581, 1.99985989]]), array([[6.12452178, 2.81777287, 4.73687043, 1.5826772 ]])]
mus: [array([[5.22473808, 3.20540036, 2.25413618, 0.56096679]]), array([[6.74339367, 3.0221281 , 5.60740602, 2.0218882 ]]), array([[6.10845561, 2.82855564, 4.69352419, 1.56114087]])]
mus: [array([[5.14234956, 3.23423873, 2.02742577, 0.47033729]]), array([[6.75448259, 3.01706367, 5.65863445, 2.03943141]]), array([[6.09876893, 2.84329338, 4.64698249, 1.54191536]])]
mus: [array([[5.07657662, 3.26810793, 1.83389916, 0.39500599]]), array([[6.76321238, 3.01249413, 5.69265596, 2.0509455 ]]

In [22]:
ax = sns.distplot(df.cluster, color = 'red')

NameError: name 'df' is not defined